<a href="https://colab.research.google.com/github/yogeshchauhan20/Accredian/blob/main/llm5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q gradio sqlalchemy requests transformers torch accelerate

In [2]:
import os, json, re, time, datetime, traceback
from typing import Optional, Dict, Any, List
import requests
from sqlalchemy import create_engine, Column, Integer, Text, DateTime
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
import gradio as gr

In [3]:
DB_PATH = "/content/plans.sqlite"
REQUEST_TIMEOUT = 10

In [4]:
engine = create_engine(f"sqlite:///{DB_PATH}", connect_args={"check_same_thread": False})
Base = declarative_base()

/tmp/ipython-input-3269977398.py:2: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [5]:
class Plan(Base):
    __tablename__ = "plans"
    id = Column(Integer, primary_key=True)
    goal = Column(Text, nullable=False)
    plan_text = Column(Text, nullable=False)
    created_at = Column(DateTime, default=datetime.datetime.utcnow)

Base.metadata.create_all(bind=engine)
SessionLocal = sessionmaker(bind=engine)

In [6]:
def save_plan(goal: str, plan_text: str) -> int:
    session = SessionLocal()
    try:
        plan_record = Plan(goal=goal, plan_text=plan_text)
        session.add(plan_record)
        session.commit()
        session.refresh(plan_record)
        return plan_record.id
    finally:
        session.close()

In [7]:
def get_all_plans():
    session = SessionLocal()
    try:
        return session.query(Plan).order_by(Plan.created_at.desc()).all()
    finally:
        session.close()

In [8]:
def web_search(query: str, num_results: int = 3) -> List[Dict]:
    """Search using Wikipedia API - completely free"""
    try:
        wiki_url = "https://en.wikipedia.org/w/api.php"
        params = {
            "action": "opensearch",
            "search": query,
            "limit": num_results,
            "format": "json"
        }
        headers = {"User-Agent": "PlanGenerator/1.0"}
        response = requests.get(wiki_url, params=params, headers=headers, timeout=REQUEST_TIMEOUT)
        if response.status_code == 200:
            data = response.json()
            titles, snippets, links = data[1], data[2], data[3]
            return [{"title": t, "snippet": s, "link": l} for t, s, l in zip(titles, snippets, links)]
    except Exception as e:
        print(f"Wikipedia search error: {e}")
    return []

In [9]:
def get_weather_for_city(city: str) -> Optional[Dict]:
    """Get weather information for a city using free APIs only"""
    try:
        geo_url = "https://nominatim.openstreetmap.org/search"
        geo_params = {
            "q": city,
            "format": "json",
            "limit": 1
        }
        headers = {"User-Agent": "PlanAgent/1.0"}
        geo_response = requests.get(geo_url, params=geo_params, headers=headers, timeout=REQUEST_TIMEOUT)
        if geo_response.status_code != 200:
            return None
        geo_data = geo_response.json()
        if not geo_data:
            return None
        lat, lon = float(geo_data[0]["lat"]), float(geo_data[0]["lon"])
        weather_url = "https://api.open-meteo.com/v1/forecast"
        weather_params = {
            "latitude": lat,
            "longitude": lon,
            "current_weather": "true"
        }
        weather_response = requests.get(weather_url, params=weather_params, timeout=REQUEST_TIMEOUT)
        if weather_response.status_code == 200:
            weather_data = weather_response.json()
            current = weather_data.get("current_weather", {})
            return {
                "temperature": current.get("temperature"),
                "description": f"Wind {current.get('windspeed', 0)} km/h",
                "city": city
            }
    except Exception as e:
        print(f"Weather error: {e}")
    return None

In [10]:
def get_llm_pipeline():
    """Initialize a reliable LLM for planning tasks"""
    try:
        from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM
        import torch
        model_name = "microsoft/phi-3-mini-4k-instruct"
        device = 0 if torch.cuda.is_available() else -1
        print(f"Loading {model_name} on {'GPU' if device == 0 else 'CPU'}...")
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
        llm = pipeline(
            "text2text-generation",
            model=model,
            tokenizer=tokenizer,
            device=device
        )
        return llm
    except Exception as e:
        print(f"Error loading LLM: {e}")
        return None

In [11]:
llm_pipeline = get_llm_pipeline()

def llm_prompt(goal, search_results=None, weather_info=None):
    """Compose a richer prompt for better LLM output."""
    prompt = (
        f"You are an expert planner. "
        f"Your goal is: {goal.strip()}\n"
    )
    if weather_info:
        prompt += f"Current weather in {weather_info['city']}: {weather_info['temperature']}°C, {weather_info['description']}.\n"
    if search_results:
        prompt += "Helpful background:\n"
        for i, r in enumerate(search_results[:2], 1):
            prompt += f"Resource {i}: Title: {r['title']}. Snippet: {r['snippet']}.\n"
    prompt += (
        "Generate a detailed, step-by-step plan for the above goal. Structure your answer in steps with practical actions and relevant tips."
    )
    return prompt

Loading microsoft/phi-3-mini-4k-instruct on CPU...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

Error loading LLM: Unrecognized configuration class <class 'transformers.models.phi3.configuration_phi3.Phi3Config'> for this kind of AutoModel: AutoModelForSeq2SeqLM.
Model type should be one of BartConfig, BigBirdPegasusConfig, BlenderbotConfig, BlenderbotSmallConfig, EncoderDecoderConfig, FSMTConfig, GPTSanJapaneseConfig, GraniteSpeechConfig, LEDConfig, LongT5Config, M2M100Config, MarianConfig, MBartConfig, MT5Config, MvpConfig, NllbMoeConfig, PegasusConfig, PegasusXConfig, PLBartConfig, ProphetNetConfig, Qwen2AudioConfig, SeamlessM4TConfig, SeamlessM4Tv2Config, SwitchTransformersConfig, T5Config, T5GemmaConfig, UMT5Config, VoxtralConfig, XLMProphetNetConfig.


In [12]:
def generate_plan_with_llm(goal: str, search_results: List[Dict], weather_info: Optional[Dict]) -> str:
    """Generate plan using LLM with goal-focused instruction and fallback."""
    if llm_pipeline is None:
        return create_enhanced_fallback_plan(goal, search_results, weather_info)
    try:
        prompt_string = llm_prompt(goal, search_results, weather_info)
        response = llm_pipeline(
            prompt_string,
            max_length=256,
            min_length=64,
            temperature=0.7,
            do_sample=True,
            early_stopping=True
        )
        generated_text = response[0]["generated_text"].strip()
        if (
            len(generated_text) < 30 or
            generated_text.strip().lower()[:30] in prompt_string.strip().lower()[:120]
        ):
            return create_enhanced_fallback_plan(goal, search_results, weather_info)
        plan_lines = [f"# {goal}\n"]
        if weather_info:
            plan_lines.append(f"🌡️ **Weather:** {weather_info['temperature']}°C in {weather_info['city']}\n")
        plan_lines.append("## AI Generated Plan\n")
        plan_lines.append(generated_text)
        if search_results:
            plan_lines.append("\n### 📚 Related Resources")
            for result in search_results[:2]:
                plan_lines.append(f"- [{result['title']}]({result['link']})")
        plan_lines.append(f"\n*Generated on {datetime.datetime.now().strftime('%Y-%m-%d %H:%M')}*")
        return "\n".join(plan_lines)
    except Exception as e:
        print(f"LLM generation error: {e}")
        return create_enhanced_fallback_plan(goal, search_results, weather_info)

In [13]:
def create_enhanced_fallback_plan(goal: str, search_results: List[Dict], weather_info: Optional[Dict]) -> str:
    """Enhanced fallback with more intelligent content"""
    plan_lines = [f"# {goal}\n"]
    if weather_info:
        plan_lines.append(f"🌡️ **Current Weather:** {weather_info['temperature']}°C in {weather_info['city']}\n")
    plan_lines.append("## Structured Plan\n")
    goal_lower = goal.lower()
    if "vegetarian food tour" in goal_lower and "hyderabad" in goal_lower:
        plan_lines.extend([
            "### Day 1: Traditional Vegetarian Cuisine",
            "- **Morning (9-11 AM):** Visit Paradise Food Court for South Indian breakfast",
            "- **Afternoon (1-3 PM):** Explore Chutneys restaurant for authentic Andhra thali",
            "- **Evening (6-8 PM):** Street food tour in Abids area - pani puri, bhel puri",
            "",
            "### Day 2: Modern & Traditional Mix",
            "- **Morning (9-11 AM):** Breakfast at Minerva Coffee Shop - famous dosas",
            "- **Afternoon (1-3 PM):** Lunch at Govind Dosa for variety of dosas",
            "- **Evening (6-8 PM):** Dinner at Ohri's Jiva Imperia for upscale vegetarian dining",
            ""
        ])
    elif "study routine" in goal_lower and "python" in goal_lower:
        plan_lines.extend([
            "**Step 1:** Set up development environment (VS Code, Python 3.9+)",
            "**Step 2:** Daily practice schedule - 2 hours morning coding",
            "**Step 3:** Learn basics: variables, loops, functions (Week 1-2)",
            "**Step 4:** Projects: build small applications (Week 3-4)",
            "**Step 5:** Regular review and practice coding challenges",
            ""
        ])
    elif "weekend" in goal_lower and "vizag" in goal_lower:
        plan_lines.extend([
            "### Saturday: Beach & Nature",
            "- **Morning:** Visit RK Beach and Submarine Museum",
            "- **Afternoon:** Kailasagiri Hill Park for panoramic views",
            "- **Evening:** Beach walk and seafood dinner at Sea Inn",
            "",
            "### Sunday: Culture & Adventure",
            "- **Morning:** Araku Valley day trip (early start)",
            "- **Afternoon:** Coffee plantations and tribal museum",
            "- **Evening:** Return to city, relax at hotel",
            ""
        ])
    else:
        plan_lines.extend([
            "### Phase 1: Preparation & Research",
            "- Gather necessary resources and information",
            "- Set clear objectives and timeline",
            "- Identify potential challenges and solutions",
            "",
            "### Phase 2: Implementation",
            "- Execute planned activities step by step",
            "- Monitor progress and make adjustments",
            "- Stay flexible and adapt as needed",
            "",
            "### Phase 3: Review & Optimize",
            "- Evaluate results against objectives",
            "- Document lessons learned",
            "- Plan improvements for future iterations",
            ""
        ])
    if search_results:
        plan_lines.append("### 📚 Additional Resources")
        for result in search_results[:2]:
            plan_lines.append(f"- [{result['title']}]({result['link']})")
        plan_lines.append("")
    plan_lines.append(f"*Generated on {datetime.datetime.now().strftime('%Y-%m-%d %H:%M')} (Enhanced Fallback)*")
    return "\n".join(plan_lines)

In [14]:
def create_plan(goal: str) -> str:
    if not goal or not goal.strip():
        return "Please enter a valid goal."
    cities = ["Hyderabad", "Jaipur", "Vizag", "Visakhapatnam", "Delhi", "Mumbai",
              "Bangalore", "Chennai", "Kolkata", "Pune", "Goa"]
    found_city = None
    for city in cities:
        if city.lower() in goal.lower():
            found_city = city
            break
    search_query = goal[:50]
    search_results = web_search(search_query, num_results=3)
    weather_info = None
    if found_city:
        weather_info = get_weather_for_city(found_city)
    plan_text = generate_plan_with_llm(goal, search_results, weather_info)
    plan_id = save_plan(goal, plan_text)
    plan_text += f"\n*Saved as Plan ID: {plan_id}*"
    return plan_text

In [15]:
def generate_plan_interface(goal: str):
    if not goal:
        return "Please enter a goal to generate a plan."
    try:
        return create_plan(goal)
    except Exception as e:
        return f"❌ Error generating plan: {str(e)}"

In [16]:
def show_plan_history():
    plans = get_all_plans()
    if not plans:
        return "No previous plans found."
    history_text = ["# Previous Plans\n"]
    for plan in plans[:10]:
        history_text.append(f"## Plan #{plan.id} - {plan.created_at.strftime('%Y-%m-%d %H:%M')}")
        history_text.append(f"**Goal:** {plan.goal}")
        history_text.append("**Plan:**")
        plan_lines = plan.plan_text.split('\n')[:5]
        for line in plan_lines:
            history_text.append(line)
        history_text.append("...\n---\n")
    return "\n".join(history_text)

In [17]:
def create_gradio_app():
    with gr.Blocks(title="AI Plan Generator", theme=gr.themes.Soft()) as app:
        gr.Markdown("# 🎯 AI Plan Generator")
        gr.Markdown("Enter your goal and get a structured plan with web research and weather context!")
        with gr.Tab("Create New Plan"):
            with gr.Row():
                with gr.Column():
                    goal_input = gr.Textbox(
                        label="Your Goal",
                        placeholder="e.g., Plan a 2-day vegetarian food tour in Hyderabad",
                        lines=3
                    )
                    generate_btn = gr.Button("Generate Plan", variant="primary", size="lg")
                with gr.Column():
                    gr.Markdown("""
                    ### Example Goals:
                    - Plan a 2-day vegetarian food tour in Hyderabad
                    - Organize a 5-step daily study routine for learning Python
                    - Create a weekend plan in Vizag with beach, hiking, and seafood
                    - Design a workout routine for beginners
                    """)
            plan_output = gr.Markdown(label="Generated Plan")
            generate_btn.click(generate_plan_interface, inputs=[goal_input], outputs=[plan_output])
        with gr.Tab("Previous Plans"):
            history_output = gr.Markdown()
            refresh_btn = gr.Button("Refresh History")
            refresh_btn.click(show_plan_history, outputs=[history_output])
    return app

In [18]:
if __name__ == "__main__":
    print("Starting Plan Generator with LLM...")
    print("Loading model... this may take a moment on first run.")
    app = create_gradio_app()
    app.launch(share=True, server_name="0.0.0.0", server_port=7879)

Starting Plan Generator with LLM...
Loading model... this may take a moment on first run.
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://2cd355285728dd0a6d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [19]:
!jupyter nbconvert --ClearMetadataPreprocessor.enabled=True --to notebook --inplace untitled5.ipynb

[NbConvertApp] WARNING | pattern 'untitled5.ipynb' matched no files
This application is used to convert notebook files (*.ipynb)
        to various other formats.


Options
The options below are convenience aliases to configurable class-options,
as listed in the "Equivalent to" description-line of the aliases.
To see all configurable class-options for some <cmd>, use:
    <cmd> --help-all

--debug
    set log level to logging.DEBUG (maximize logging output)
    Equivalent to: [--Application.log_level=10]
--show-config
    Show the application's configuration (human-readable format)
    Equivalent to: [--Application.show_config=True]
--show-config-json
    Show the application's configuration (json format)
    Equivalent to: [--Application.show_config_json=True]
--generate-config
    generate default config file
    Equivalent to: [--JupyterApp.generate_config=True]
-y
    Answer yes to any questions instead of prompting.
    Equivalent to: [--JupyterApp.answer_yes=True]
--execute
    E